In [ ]:
from train import *
from utils import *
from data import *
import json
import numpy as np 
import networkx as nx
import os
import csv 
max_nodes = 15
name = 'YELP'
## Loading the main graph 
# graphs = create_graphs.create(args)
print('Loading graph dataset: '+str(name))
G = nx.Graph()

path = 'dataset/'+name+'/'
data_ids_file = open(path+name+'_user_id_map_inverse.txt','r')
data_ids = json.load(data_ids_file)
data_ids_map_file = open(path + name + '_user_id_map_inverse.txt','r')
data_ids_map = json.load(data_ids_map_file)
data_adj = np.loadtxt(path+name+'_A.txt', delimiter=',').astype(int)
data_node_att = np.loadtxt(path+name+'_node_attributes.txt', delimiter=',')
data_node_label = np.loadtxt(path+name+'_node_labels.txt', delimiter=',').astype(int)
data_graph_indicator_org = np.loadtxt(path+name+'_graph_indicator.txt', delimiter=',').astype(int)
data_graph_labels = np.loadtxt(path+name+'_graph_labels.txt', delimiter=',').astype(int)
data_tuple = list(map(tuple, data_adj))

# create graph-
G.add_edges_from(data_tuple)
graph_num = data_graph_indicator_org.max()
count_id = 0
data_graph_indicator = []
graph_num_list = [0 for _ in range(graph_num)]
for i in range(data_node_label.shape[0]):
    node_group_id = data_graph_indicator_org[i]
    if graph_num_list[node_group_id-1] < max_nodes:
        graph_num_list[node_group_id-1] += 1
        G.add_node(count_id+1, feature = data_node_att[i])
        G.add_node(count_id+1, label = data_node_label[i])
        G.add_node(count_id+1, user_id = data_ids[str(i+1)])
        G.add_node(count_id+1, group_id = data_graph_indicator_org[i])
        data_graph_indicator.append(data_graph_indicator_org[i])
        count_id += 1
G.remove_nodes_from(list(nx.isolates(G)))
data_graph_indicator = np.array(data_graph_indicator)
node_list = np.arange(count_id) + 1
graphs = []
total_num_of_nodes = 0
for i in range(graph_num):    
    # find the nodes for each graph
    nodes = node_list[data_graph_indicator==i+1]
    if len(nodes)>max_nodes:
        nodes = nodes[0:max_nodes]
    total_num_of_nodes += len(nodes)
    G_sub = G.subgraph(nodes)
    G_attributes = nx.get_node_attributes(G_sub,'feature')
    G_sub.graph['label'] = data_graph_labels[i]
    graphs.append(G_sub)
    G_ids = nx.get_node_attributes(G_sub,'user_id')
#     print(G_ids)
#     if G_sub.number_of_nodes() > max_nodes:
#         max_nodes = G_sub.number_of_nodes()
# shuffling the data 
# print(total_num_of_nodes)

graphs_len = len(graphs)
# random.seed(123)
# graphs_list = list(enumerate(graphs))
# shuffle(graphs_list)
# graphs_idx, graphs = zip(*graphs_list)

graphs_test = graphs[int(0.8 * graphs_len):]
graphs_test_len = len(graphs_test)
graphs_test_dic = {}
nodes_test_len = 0
test_id = 0
for graph in graphs_test:
    node_ids = nx.get_node_attributes(graph,'user_id')
    for key, value in node_ids.items():
        test_id += 1
        graphs_test_dic[value] = test_id
        nodes_test_len += 1
    
graphs_train = graphs[0:int(0.8*graphs_len)]
# graphs_train_idx = graphs_idx[0:int(0.8*graphs_len)]
graphs_train_len = len(graphs_train)
graphs_train_dic = {}
nodes_train_len = 0
train_id = 0
for graph in graphs_train:
    node_ids = nx.get_node_attributes(graph,'user_id')
    for key, value in node_ids.items():
        train_id += 1 
        graphs_train_dic[value] = train_id
        nodes_train_len += 1
# print(graphs_train_dic)
graphs_validate = graphs[0:int(0.2*graphs_len)]

num_of_intervals = 0
dynamic_graphs_path = path + 'dynamic_graphs/'
for _, dirnames,_ in os.walk(dynamic_graphs_path):
    num_of_intervals += len(dirnames)
print(num_of_intervals)
num_of_groups = []


# Train data extraction
train_data = []
for i in range(num_of_intervals-1):
    train_adj_mat = np.zeros((nodes_train_len,max_nodes))
    read_path = dynamic_graphs_path + str(i+1) + '/'
    data_ids_file = open(read_path + name +'_user_id_map.txt','r')
    data_ids_dyn = json.load(data_ids_file)
    data_ids_inverse_dyn = dict([(value, key) for key, value in data_ids_dyn.items()]) 
    data_adj_dyn = np.loadtxt(read_path+name+'_A.txt', delimiter=',').astype(int)
    data_graph_indicator_dyn = np.loadtxt(read_path+name+'_graph_indicator.txt', delimiter=',').astype(int)  
    if len(data_adj_dyn) != 0:
        num_of_groups.append(len(np.unique(data_adj_dyn)))
    else:
        num_of_groups.append(0)
    if len(data_adj_dyn) != 0:
        data_tuple_dyn = list(map(tuple, data_adj_dyn))
        for tuple_item in data_tuple_dyn:
            main_idx = tuple_item[1]
            neighbor_idx = tuple_item[0]
            group_id = data_graph_indicator_dyn[main_idx-1]
#             if group_id in graphs_train_idx:
            if group_id < graphs_train_len:
#                 print(group_id, graphs_train_idx.index(group_id))
                user_id_main = data_ids_inverse_dyn[main_idx]
                user_id_neighbor = data_ids_inverse_dyn[neighbor_idx]
#                 group_idx = graphs_train_idx.index(group_id)
                user_group = graphs_train[group_id - 1]
                for key,value in graphs_train_dic.items():
                    if key == user_id_main:                        
                        main_user_idx = value
                        break
                        
                user_ids = nx.get_node_attributes(user_group,'user_id')                
                counter = 0
                for key,value in user_ids.items():
                    if user_ids[key] == user_id_neighbor and counter<max_nodes:                        
                        train_adj_mat[main_user_idx-1][counter] = 1
                        break
                    counter += 1
        train_data.append(train_adj_mat)

#     for m in range(len(train_adj_mat)):
#         for n in range(len(train_adj_mat[0])):
#             if train_adj_mat[m][n] == 1:
#                 print(i,' ',m,' ',n)
                
# Test data extraction
test_data = []
for i in range(num_of_intervals-1):
    test_adj_mat = np.zeros((nodes_test_len,max_nodes))
    read_path = dynamic_graphs_path + str(i+1) + '/'
    data_ids_file = open(read_path + name +'_user_id_map.txt','r')
    data_ids_dyn = json.load(data_ids_file)
    data_ids_inverse_dyn = dict([(value, key) for key, value in data_ids_dyn.items()]) 
    data_adj_dyn = np.loadtxt(read_path+name+'_A.txt', delimiter=',').astype(int)
    data_graph_indicator_dyn = np.loadtxt(read_path+name+'_graph_indicator.txt', delimiter=',').astype(int)  
    if len(data_adj_dyn) != 0:
        num_of_groups.append(len(np.unique(data_adj_dyn)))
    else:
        num_of_groups.append(0)
    if len(data_adj_dyn) != 0:
        data_tuple_dyn = list(map(tuple, data_adj_dyn))
        for tuple_item in data_tuple_dyn:
            main_idx = tuple_item[1]
            neighbor_idx = tuple_item[0]
            group_id = data_graph_indicator_dyn[main_idx-1]
#             if group_id in graphs_train_idx:
            if group_id >= graphs_train_len:
#                 print(group_id, graphs_train_idx.index(group_id))
                user_id_main = data_ids_inverse_dyn[main_idx]
                user_id_neighbor = data_ids_inverse_dyn[neighbor_idx]
#                 group_idx = graphs_train_idx.index(group_id)
                user_group = graphs[group_id - 1]
                for key,value in graphs_test_dic.items():
                    if key == user_id_main:                        
                        main_user_idx = value
                        break
                        
                user_ids = nx.get_node_attributes(user_group,'user_id')                
                counter = 0
                for key,value in user_ids.items():
                    if user_ids[key] == user_id_neighbor and counter<max_nodes:                        
                        test_adj_mat[main_user_idx-1][counter] = 1
                        break
                    counter += 1
        test_data.append(test_adj_mat)

#     for m in range(len(test_adj_mat)):
#         for n in range(len(train_adj_mat[0])):
#             if train_adj_mat[m][n] == 1:
#                 print(i,' ',m,' ',n)


    
# with open('output.csv','w') as result_file:
#     wr = csv.writer(result_file, dialect='excel')
#     wr.writerow(num_of_groups)               
                    
                    
            
    
    



In [ ]:
import torch
import torch.nn
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
total_epoch = 10
current_epoch = 0
time_steps = len(train_data)-1
batch_size = 16
hidden_size = max_nodes
# train the lstm
train_data_transposed = []
for adj_mat in train_data:
    train_data_transposed.append(np.transpose(adj_mat))
train_data_seg = train_data[0:-1]
target_seg = train_data[-1]
train_batch_number = int(graphs_train_len/batch_size)
train_data_np = np.array(train_data_seg)
target_np = np.array(target_seg)
many_one_model = nn.LSTM(max_nodes,hidden_size)
while current_epoch < total_epoch:
    for i in range(train_batch_number):        
        input_data = train_data_np[:,i*train_batch_number:i*train_batch_number+batch_size,:]
        target = target_np[i*train_batch_number:i*train_batch_number+batch_size,:]
        input_data = Variable(torch.from_numpy(input_data).float())
        target = Variable(torch.from_numpy(target).float())
        predicted_output,_ = many_one_model(input_data)
        last_output = predicted_output[-1,:,:]
        print(last_output, ' ',target)    
        error = nn.functional.binary_cross_entropy_with_logits(last_output,target)
        error.backward()
    current_epoch += 1


In [ ]:
# import torch
# # c = [3, 5, 7, 9]
# # v=list(enumerate(c))
# # random.shuffle(v)
# # c_idx,c = zip(*v)
# # print(c_idx,' ',c)
# train_data_np = np.array(train_data)
# train_sample = train_data_np[-1,:,:]
# input_seq = Variable(torch.tensor(train_sample))
# print(input_seq)
time_steps = 10
batch_size = 3
in_size = 5
classes_no = 7
model = nn.LSTM(in_size,hidden_size = 1)
input_seq = Variable(torch.randn(time_steps, batch_size, in_size))
print(input_seq.size(1))
# target = Variable(torch.LongTensor(batch_size).random_(0, 1))
target = Variable(input_seq[-1])
output_seq, _ = model(input_seq)
last_output = output_seq[-1]
print(last_output, ' ',target)
err = nn.functional.binary_cross_entropy(last_output, target)
